# Demo of CCC for PSL Meetup

This notebook provides a demonstration of the Cost-of-Capital-Calculator (CCC) for the PSL Meetup on April 29, 2019.

To run this notebook, you will need to follow the instructions to install CCC as described in the CCC README [here](https://github.com/PSLmodels/Cost-of-Capital-Calculator).  In particular, you need to:

* Install the [Anaconda distribution](https://www.anaconda.com/distribution/) of Python
* Install the CCC package by typing `conda install -c pslmodels ccc` in the command prompt.

Once you follow the above, you will be ready to work with this Jupyter Notebook.

## First things first, import necessary packages

In [3]:
# import packages
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# import CCC classes that we'll work with
from ccc.data import Assets
from ccc.parameters import Specification, DepreciationParams
from ccc.calculator import Calculator
# to print bokeh plots inline
output_notebook()

Loading BokehJS ...

## Create an instance of the Assets class

This is the class that contains the data that underlie CCC.  The basic object is a Pandas DataFrame where each row represents a combination of a specific type of asset, industry (approximately 6-digit NAICS) and tax treatment.  The columns represent names and codes for the asset and industry classifications, the tax depreciation rules used for that asset, and the rate of economic depreciation (`delta`).

In [4]:
assets = Assets()
assets.df.head(n=5)

,Unnamed: 0,tax_treat,assets,bea_asset_code,bea_ind_code,Industry,minor_code_alt,major_asset_group,minor_asset_group,major_industry,asset_name,delta
0,0,corporate,0.0,RD32,110C,Farms,111,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
1,1,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",113,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
2,2,corporate,0.0,RD32,113F,"Forestry, fishing, and related activities",114,Intellectual Property,Intellectual Property,"Agriculture, forestry, fishing, and hunting",Aerospace products and parts manufacturing,0.22
3,3,corporate,0.0,RD32,2110,Oil and gas extraction,211110,Intellectual Property,Intellectual Property,Mining,Aerospace products and parts manufacturing,0.22
4,4,corporate,0.0,RD32,2120,"Mining, except oil and gas",212110,Intellectual Property,Intellectual Property,Mining,Aerospace products and parts manufacturing,0.22


## Create instances of the two parameters classes

The `Specification` class contains many of the model parameters, although depreciation system parameters are contained in the `DepreciationParams` class.  Both are required arguments for the `Calculator` object.

A `Specification` object has methods that load the data from a file that contains the default parameter values (`default_parameters.json`) and then stores them as attributes of the `Specification` class object.  

`p.u` looks into the instance of this class named `p` and executing the cell below will show you that `p.u` is dictionary containtin marginal tax rates on corporate and pass-through income.

Note that the `Specification` class has some arguments, such as `call_tc`, which defaults to `False`, but if set to `True` will call the Tax-Calculator to estimate marginal tax rates on individual filers' income.

Similar to `Specification`, `DepreciationParams` loads default parameters from a JSON file, in this case the file `tax_depreciation_rules.json`.

In [5]:
# Create an instance of the Specification class
# p = Specification()
p = Specification(call_tc=True)
# Look at attributes
p.u

year:  2020
{'tau_nc': array([0.20321129]), 'tau_div': array([0.19085268]), 'tau_int': array([0.31484783]), 'tau_scg': array([0.29355133]), 'tau_lcg': array([0.19028709]), 'tau_td': array([0.22067719]), 'tau_h': array([0.04387959])}


{'c': array([0.21]), 'nc': array([0.20321129])}

In [6]:
# Create an instance of the DepreciationParams class
dp = DepreciationParams()
# Look at an attribute of the dp object
dp.asset[0]

{'year': 2020,
 'GDS_life': 5.0,
 'minor_asset_group': 'Computers and Software',
 'value': {'life': 5, 'method': 'DB 200%'},
 'ADS_life': 5.0,
 'major_asset_group': 'Equipment',
 'system': 'GDS',
 'asset_name': 'Mainframes',
 'BEA_code': 'EP1A'}

## Create an instance of the Calculator class

This class does the calculations on the data.  It takes as arguments the data object (named `assets` here) and the parameters object (named `p` here).

Excuting the cell below creates an instance of the calculator class with these data and parameters, but does not yet excecute any calculations.

In [7]:
# Create an instance of the Calculator class
calc1 = Calculator(p, dp, assets)

With an instance of the `Calculator` class created, we can start doing some calculations with these data.

For instance, we can compute a table of the share of corporate vs non-corporate assets across each industry.    

In [8]:
# Look at shares of assets across industry
calc1.asset_share_table()

,Industry,Corporate,Pass-Through
0,"Agriculture, forestry, fishing, and hunting",0.309688,0.690312
1,Mining,0.826839,0.173161
2,Utilities,0.948540,0.051460
3,Construction,0.286840,0.713160
4,Manufacturing,0.859014,0.140986
5,Wholesale trade,0.715166,0.284834
6,Retail trade,0.728628,0.271372
7,Transportation and warehousing,0.789652,0.210348
8,Information,0.918722,0.081278
9,Finance and insurance,0.786056,0.213944


## Specifying a reform policy

To see some more interesting results, we will want to create another `Calculator` object with a change in policy or economic assumptions.  We can do this in a way analogous to our original instantiation of the `Calculator` object above.  In particular, we'll need to create a new `Specification` object (we can use the same underlying data, which was in the object we named `assets`).

In the code below, we'll specify our "reform" as current law tax policy for 2026 (the baseline parameters above came from the default model year, 2019).  In addition, we'll increase the corporate income tax rate from 21 to 25%.

In [9]:
# Create another policy
p2 = Specification(year=2026)
p2.update_specification({'CIT_rate': 0.35})
calc2 = Calculator(p2, dp, assets)

## Tabular output

Now with two `Calculator` objects named `calc1` and `calc2` (representing the baseline and reform policies), we are ready to compute some of the changes in effective tax rates, cost of capital, or other variables measured in this model.

We start with an overall summary table showing the marginal effective total tax rates (METTRs) for all investments, corporate investments, and pass-through investments under varying financing assumptions.  This is done through the `summary_table` function.  It takes a calculator object as an argument.

In [10]:
# Look at differences in METTRs between the two policies
calc1.summary_table(calc2) # calc1 is the baseline, calc2 the reform

,,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.073237,25.995323,7.922086
1,Corporations,18.600704,29.056826,10.456122
2,Equity Financed,21.644981,35.386666,13.741685
3,Debt Financed,7.260980,-1.669848,-8.930828
4,Pass-Through Entities,18.982667,23.444056,4.461389
5,Equity Financed,17.206830,22.972001,5.765171
6,Debt Financed,25.767874,25.503277,-0.264597


the `Calculator.summary_table()` method defaults to showing the results for the METTR, but there is a keyword argument that would allow you to view the output for other variables computed in CCC.  We can use this to see changes in the cost of capital (denoted by $\rho$ in the model):

In [11]:
# Look at how cost of capital changed
calc1.summary_table(calc2, output_variable='rho')

,,Cost of Capital Under Baseline Policy,Cost of Capital Under Reform Policy,Change from Baseline (pp)
0,Overall,5.622563,6.224449,0.601887
1,Corporations,5.658997,6.493061,0.834064
2,Equity Financed,6.571207,7.968742,1.397536
3,Debt Financed,3.723999,3.396877,-0.327121
4,Pass-Through Entities,5.579133,5.904264,0.325131
5,Equity Financed,6.218955,6.684414,0.465459
6,Debt Financed,4.015211,4.000950,-0.014261


One can also save results to disk by specifying an output type ('excel', 'json', 'csv', 'tex') and a file path: 

In [12]:
# Save these results to disk
calc1.summary_table(calc2, output_variable='rho', output_type='excel', path='cc_table.xlsx')

There are also `Calculator` methods to compute summary tables by asset type or industry.  These are computed in the next two cells.

In [13]:
# Summary by asset type
calc1.asset_summary_table(calc2)

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.073237,25.995323,7.922086
1,Corporate,18.600704,29.056826,10.456122
2,Equipment,7.706938,23.302431,15.595493
3,Structures,19.242489,28.612810,9.370320
4,Intellectual Property,7.706938,12.179073,4.472136
5,Inventories,29.951914,40.575483,10.623569
6,Land,27.088481,36.543121,9.454640
7,Pass-through,18.982667,23.444056,4.461389
8,Equipment,2.119986,15.015279,12.895293
9,Structures,17.608806,21.552566,3.943760


In [14]:
# Summary by industry
calc1.industry_summary_table(calc2)

,Category,Marginal Effective Total Tax Rate Under Baseline Policy,Marginal Effective Total Tax Rate Under Reform Policy,Change from Baseline (pp)
0,Overall,18.073237,25.995323,7.922086
1,Corporate,18.600704,29.056826,10.456122
2,"Agriculture, forestry, fishing, and hunting",19.458235,31.606254,12.148018
3,Mining,10.642356,18.814935,8.172579
4,Utilities,8.717925,20.936996,12.219071
5,Construction,19.990478,31.337689,11.347211
6,Manufacturing,18.893192,27.869133,8.975941
7,Wholesale trade,24.278532,35.206596,10.928064
8,Retail trade,25.255507,35.616880,10.361373
9,Transportation and warehousing,11.907455,21.523209,9.615753


## Visualizations

If one wants to visualize the effects of changes in tax policy, the `Calculator` class has a few methods for this.  

We can use the `grouped_bar` method to show differential effects across assets (the default):

In [15]:
# Visualizing changes by asset
aplot = calc1.grouped_bar(calc2)
show(aplot)

or we can change from the default asset category split to a split by industry by changing the value of the `group_by_asset` keyword argument:

In [16]:
# Visualizing changes by industry
iplot = calc1.grouped_bar(calc2, group_by_asset=False)
show(iplot)

There's also a plot that illustrates the range of the effects of taxes on investments across asset types, by showing the min, max, and mean values:

In [17]:
# Plot to show variation in METTRs across assets
rplot = calc1.range_plot(calc2, output_variable='metr')
show(rplot)

changing to show the effects on pass-through businesses and for a different output variable:

In [18]:
# Plot to show variation in METTRs across assets
rplot2 = calc1.range_plot(calc2, corporate=False, output_variable='mettr')
show(rplot2)

There's a bubble plot too, though it doesn't yet allow for many options (currently only plots METTRs for corporate entities):

In [19]:
bplot = calc1.asset_bubble(calc2)
show(bplot)

## Summary

This notebook provides a brief example of how one would work with CCC.  Please explore the source code to see additional flexiblity in the functions.  And please leave any questions or suggestions in the CCC repo at [https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues](https://github.com/PSLmodels/Cost-of-Capital-Calculator/issues).